# Create LM

In [1]:
import httpx
from typing import List, Optional
import brokit as bk
class Llama(bk.LM):
    def __init__(self, model_name: str, base_url:str = "http://localhost:11434", temperature:float=0.0, top_p:float=1.0, seed:int=55, **kwargs):
        super().__init__(model_name=model_name, model_type=bk.ModelType.CHAT)
        self.base_url = base_url
        self.client = httpx.Client(timeout=60.0)  # Reusable client
        self.model_params = {
            "temperature": temperature,
            "top_p": top_p,
            "seed": seed,
            **kwargs
        }

    def request(self, prompt:Optional[str]=None, messages:Optional[List[bk.Message]]=None, images:Optional[List[bk.Image]]=None, audios=None, **kwargs) -> dict:
        url = f"{self.base_url}/api/chat"
        params = {**self.model_params, **kwargs}
        if messages is not None:
            _messages = [msg.to_dict() if isinstance(msg, bk.Message) else msg for msg in messages]
        else:
            _messages = [{"role": "user", "content": prompt}]
        if images:
            _messages[-1]["images"] = [image.to_base64() for image in images]
        request_params = {
            "model": self.model_name,
            "messages": _messages,
            "stream": False,
            "options": {**params},
        }
        response = self.client.post(
            url,
            json=request_params
        )                
        return response.json()

    def response(self, original_response: dict) -> bk.ModelResponse:
        message = original_response["message"]
        input_tokens = original_response.get("prompt_eval_count", 0)
        output_tokens = original_response.get("eval_count", 0)
        return bk.ModelResponse(
            model_name=self.model_name,
            model_type=self.model_type,
            response=message["content"],
            usage=bk.Usage(input_tokens=input_tokens, output_tokens=output_tokens),
            metadata=None
        )

# Pure QA

In [2]:
class QA(bk.Prompt):
    """Answer a question in bro-like manner"""
    question:str = bk.InputField(description="a user's question")
    answer:str = bk.OutputField(description="answer to the question")

shots = [
    bk.Shot(QA, question="What is the capital of France?", answer="Bro, it's Paris. Easy one, dude."),
    bk.Shot(QA, question="How does photosynthesis work?", answer="Yo bro, plants take sunlight and turn it into food. Science is lit, man."),
    bk.Shot(QA, question="What's the speed of light?", answer="Bruhhh, it's like 300,000 km/s. Fast as heck, bro."),
    bk.Shot(QA, question="Who invented the telephone?", answer="That's Alexander Graham Bell, my dude. Legend move right there."),
    bk.Shot(QA, question="What's the largest ocean?", answer="Pacific Ocean, bro. It's massive, no cap."),
]

llama = Llama(model_name="gemma3:12b")
qa = bk.Predictor(prompt=QA, lm=llama, shots=shots)
response = qa(question="What is the capital of France")
response

Prediction(
    answer="Paris, bro. It's the City of Lights, ya know?"
)

In [3]:
llama.history[0].request

[{'role': 'system',
  'content': "Your input fields are:\n1. question (<class 'str'>): a user's question\nYour output fields are:\n1. answer (<class 'str'>): answer to the question\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||answer||>\n{answer}\n\n<||completed||>\nIn adhering to this structure, your objective is: \nAnswer a question in bro-like manner"},
 {'role': 'user', 'content': '<||question||>\nWhat is the capital of France?'},
 {'role': 'assistant',
  'content': "<||answer||>\nBro, it's Paris. Easy one, dude.\n\n<||completed||>"},
 {'role': 'user', 'content': '<||question||>\nHow does photosynthesis work?'},
 {'role': 'assistant',
  'content': '<||answer||>\nYo bro, plants take sunlight and turn it into food. Science is lit, man.\n\n<||completed||>'},
 {'role': 'user', 'content': "<||question||>\nWhat's the speed of light?"},
 {'role': 'assistant',
  'content': "<||answer||>\nBruhhh, it's 

# ChainOfThoughts QA

In [4]:
class COT_QA(bk.Prompt):
    """Answer a question in bro-like manner"""
    question:str = bk.InputField(description="a user's question")
    reasoning:str = bk.OutputField(description="reasons why you answer this way")
    answer:str = bk.OutputField(description="answer to the question")

shots = [
    bk.Shot(COT_QA, question="What is the capital of France?", answer="Bro, it's Paris. Easy one, dude."),
    bk.Shot(COT_QA, question="How does photosynthesis work?", answer="Yo bro, plants take sunlight and turn it into food. Science is lit, man."),
    bk.Shot(COT_QA, question="What's the speed of light?", answer="Bruhhh, it's like 300,000 km/s. Fast as heck, bro."),
    bk.Shot(COT_QA, question="Who invented the telephone?", answer="That's Alexander Graham Bell, my dude. Legend move right there."),
    bk.Shot(COT_QA, question="What's the largest ocean?", answer="Pacific Ocean, bro. It's massive, no cap."),
]

llama = Llama(model_name="gemma3:12b")
cot_qa = bk.Predictor(prompt=COT_QA, lm=llama, shots=shots)
response = cot_qa(question="What is the capital of France?")
response

Prediction(
    reasoning='The question asks for the capital of France. France is a country in Europe, and its capital city is widely known and a common piece of geographical knowledge.',
    answer="Paris, bro. It's the city of lights and all that jazz."
)

In [5]:
llama.history[0].request

[{'role': 'system',
  'content': "Your input fields are:\n1. question (<class 'str'>): a user's question\nYour output fields are:\n1. reasoning (<class 'str'>): reasons why you answer this way\n2. answer (<class 'str'>): answer to the question\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||reasoning||>\n{reasoning}\n\n<||answer||>\n{answer}\n\n<||completed||>\nIn adhering to this structure, your objective is: \nAnswer a question in bro-like manner"},
 {'role': 'user', 'content': '<||question||>\nWhat is the capital of France?'},
 {'role': 'assistant',
  'content': "<||reasoning||>\nIntentionally left blank.\n\n<||answer||>\nBro, it's Paris. Easy one, dude.\n\n<||completed||>"},
 {'role': 'user', 'content': '<||question||>\nHow does photosynthesis work?'},
 {'role': 'assistant',
  'content': '<||reasoning||>\nIntentionally left blank.\n\n<||answer||>\nYo bro, plants take sunlight and turn it into fo